# Northern Brooklyn KSI Study

#### Output should be table with all the streets within region and KSI statistics in that region


####### Steps: 

####### 1. Convert the shapefile into a table with geometry
####### 2. Spatial join the shapefile with nysdot_all injury data
####### 3. Retrieve table with injury severity statistics that overlap with selected region


In [2]:
from ris import db2
from ris import pg_import_export_shps as shp
from IPython.display import clear_output
import datetime
from functools import reduce
import pandas as pd
from pandas import DataFrame
from pandas import ExcelWriter
import numpy as np
import os


clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
print 'Notebook run: {}'.format(timestamp)

# %load_ext sql 

Notebook run: 2019-06-14 10:25


In [ ]:
#db = db2.PostgresDb('dotdevpgsql02', 'GISGRID', quiet = True)
db = db2.PostgresDb('dotdevpgsql02', 'CRASHDATA', quiet = True)
#exporting study area to pg database
shp.import_shp_to_pg('C:\Users\soge\Desktop\Jupyter\Shps\Northern_Brooklyn_Study_Area\Study_Area.shp', db, schema='working', precision=False, 
                     permission=True, gdal_data=r"C:\Program Files (x86)\GDAL\gdal-data") 


User name (CRASHDATA):soge
Password (CRASHDATA)········


In [80]:
#DONE FROM CRASHDATABASE

"""
pgdb = db2.PostgresDb('dotdevpgsql02', 'CRASHDATA', quiet = True) #data connection
qry = (db2.query_to_table(pgdb, """
                                select inj as severity, count(inj) as cnt
                                from (
                                SELECT b.ext_of_inj inj, b.geom
                                FROM working.study_area a 
                                join public.nysdot_all b
                                on ST_DWithin(a.wkb_geometry, b.geom, 2)
                                where ext_of_inj is not null
                                and (ext_of_inj) like '%A%'
                                ) c
                                group by inj

                            """)
              )
print(qry)


for i,j in zip(qry.severity,qry.cnt):
    print(i,j)

for i in qry.severity:
    print i
    


count=0
for i in qry.severity:
    if 'A' in i:
        print i, i.count('A')
        count+=1
print(count)

for i in qry.cnt:
    print i

for i,j in zip(qry.severity, qry.cnt):
    if 'A' in i:
        print(i,j,j*i.count('A'))

sev_sum = 0 
for i,j in zip(qry.severity, qry.cnt):
    if 'A' in i:
        #print(i,j,j*i.count('A'))
        sev_sum+=j*i.count('A')
print(sev_sum)

df.to_csv('example.csv',index=False)

"""

User name (CRASHDATA):soge
Password (CRASHDATA)········
   severity  cnt
0         A  535
1        AA   21
2       AAA    5
3     AAAAA    1
4     AAACC    2
5       AAB    1
6      AABC    1
7       AAC    2
8        AB    7
9       ABB    3
10      ABC    3
11    ABCCC    2
12       AC   29
13      ACA    1
14     ACBB    1
15     ACBC    1
16      ACC    6
17     ACCC    6
18    ACXCB    1
19     ACXX    1
20       AK    1
21       BA    1
22     BBBA    1
23       CA    4
24     CAAA    1
25      CAB    1
26    CABCC    2
27      CAC    2
28     CACC    2
29    CACCC    1
30      CCA    1
31     CCAC    1
32       KA    1
33     KABA    1


In [3]:
pgdb = db2.PostgresDb('dotdevpgsql02', 'GISGRID', quiet = True) #data connection
qry = (db2.query_to_table(pgdb, """                                
                select inj as severity, count(inj) as cnt
                from (
                SELECT b.ext_of_inj inj, b.geom
                FROM working.study_area a 
                join public.nysdot_all b
                on ST_DWithin(st_setsrid(a.wkb_geometry, 2263), st_setsrid(ST_MakePoint(b.st_x, b.st_y),2263), 2)
                where ext_of_inj is not null
                and (ext_of_inj) like '%A%'
                and b.st_x is not null
                and b.st_y is not null
                ) c
                group by inj

                            """)
              )
print(qry)

User name (GISGRID):soge
Password (GISGRID)········
   severity  cnt
0         A  239
1        AA   22
2       AAA    4
3     AAAAA    1
4     AAAAC    1
5     AAACC    2
6       AAB    1
7      AABC    1
8       AAC    2
9        AB    4
10      ABB    4
11      ABC    3
12    ABCCC    2
13       AC   31
14     ACBC    1
15      ACC    4
16     ACCC    5
17       AK    1
18       BA    1
19     BBBA    1
20       CA    3
21    CABCC    1
22      CAC    1
23     CACC    2
24       KA    1
25     KABA    1


In [5]:
qry2 = (db2.query_to_table(pgdb, """                                
            SELECT count(b.id_), now()
            FROM working.study_area a 
            join public.fatality_nycdot_current b
            on ST_DWithin(st_setsrid(a.wkb_geometry, 2263), st_setsrid(ST_MakePoint(b.st_x, b.st_y),2263), 2)
            where b.st_x is not null
            and b.st_y is not null

                            """)
              )
print(qry2)

   count                               now
0     35  2019-06-14 10:25:19.094880-04:00


In [ ]:
#for i,j in zip(qry.severity,qry.cnt):
#    print(i,j)

#for i in qry.severity:
#    print i

#count=0
#for i in qry.severity:
#    if 'A' in i:
#        print i, i.count('A')
#        count+=1
#print(count)

#for i in qry.cnt:
#    print i

In [6]:
for i,j in zip(qry.severity, qry.cnt):
    if 'A' in i:
        print(i,j,j*i.count('A'))

('A', 535)
('AA', 21)
('AAA', 5)
('AAAAA', 1)
('AAACC', 2)
('AAB', 1)
('AABC', 1)
('AAC', 2)
('AB', 7)
('ABB', 3)
('ABC', 3)
('ABCCC', 2)
('AC', 29)
('ACA', 1)
('ACBB', 1)
('ACBC', 1)
('ACC', 6)
('ACCC', 6)
('ACXCB', 1)
('ACXX', 1)
('AK', 1)
('BA', 1)
('BBBA', 1)
('CA', 4)
('CAAA', 1)
('CAB', 1)
('CABCC', 2)
('CAC', 2)
('CACC', 2)
('CACCC', 1)
('CCA', 1)
('CCAC', 1)
('KA', 1)
('KABA', 1)


In [4]:
sev_sum = 0 
for i,j in zip(qry.severity, qry.cnt):
    if 'A' in i:
        #print(i,j,j*i.count('A'))
        sev_sum+=j*i.count('A')
print(sev_sum)

385


In [ ]:
df.to_csv('example.csv',index=False)